<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

#  Data Science and AI
## Lab 7.1.2: Random Forests

INSTRUCTIONS:

- Read the guides and hints then create the necessary analysis and code to find and answer and conclusion for the scenario below.
- The baseline results (minimum) are:
    - **Accuracy** = 0.7419
    - **ROC AUC**  = 0.6150
- Try to achieve better results!

# Foreword
It is common that companies and professionals start with the data immediately available. Although this approach works, ideally the first stp is to idenfy the problem or question and only then identify and obtain the set of data that can help to solve or answer the problem.

Also, given the current abundance of data, processing power and some particular machine learning methods, there could be a temptation to use ALL the data available. **Quality** is _**better**_ then **Quantity**!

Part of calling this discipline **Data Science** is that it is supposed to follow a process and not reach conclusions without support from evidence.

Moreover, it is a creative, exploratory, labour and iteractive processes. It is part of the process to repeat, review and change when finding a dead-end.

# Step 1: Define the problem or question
Identify the subject matter and the given or obvious questions that would be relevant in the field.

## Potential Questions
List the given or obvious questions.

## Actual Question
Choose the **one** question that should be answered.

#### Potential Questions:

 - Is there a relationship between time since first transfusion and recency of last transfusion
 
 - Can we predict who will likely transfuse in a given month with donor history?
 
#### Actual Question

 - Can we predict who will likely transfuse in a given month with donor history?

# Step 2: Find the Data
### Blood Transfusion Service Center DataSet
- **Abstract**: Data taken from the **Blood Transfusion Service Center** in Hsin-Chu City in Taiwan.
- Date Donated: 2008-10-03
- Source:
        Original Owner and Donor: Prof. I-Cheng Yeh 
        Department of Information Management 
        Chung-Hua University, 
        Hsin Chu, Taiwan 30067, R.O.C. 

- Citation Request:
    **NOTE**: Reuse of this database is unlimited with retention of copyright notice for Prof. I-Cheng Yeh and the following published paper: 

        Yeh, I-Cheng, Yang, King-Jang, and Ting, Tao-Ming, "Knowledge discovery on RFM model using Bernoulli sequence, "Expert Systems with Applications, 2008
        
### UCI - Machine Learning Repository
- Center for Machine Learning and Intelligent Systems

The [**UCI Machine Learning Repository**](http://archive.ics.uci.edu/ml/about.html) is a collection of databases, domain theories, and data generators that are used by the machine learning community for the empirical analysis of machine learning algorithms.

In [ ]:
# Find the dataset described above 
# Hint: search for it through the UCI Machine Learning Repository

# Step 3: Read the Data
- Read the data
- Perform some basic structural cleaning to facilitate the work

In [2]:
# Import libraries
import pandas as pd
import numpy as np

In [3]:
# Read data in pandas

# Check data has loaded correctly

transfusion = pd.read_csv('transfusion.data')
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


# Step 4: Explore and Clean the Data
- Perform some initial simple **EDA** (Exploratory Data Analysis)
- Check for
    - **Number of features**
    - **Data types**
    - **Domains, Intervals**
    - **Outliers** (are they valid or expurious data [read or measure errors])
    - **Null** (values not present or coded [as zero of empty strings])
    - **Missing Values** (coded [as zero of empty strings] or values not present)
    - **Coded content** (classes identified by numbers or codes to represent absence of data)

In [ ]:
# Perform EDA by investigating each of the points above 

#### Number of Features:

In [17]:
print(transfusion.shape[1])

5


#### Data Types:

In [18]:
print(transfusion.dtypes)

Recency (months)                              int64
Frequency (times)                             int64
Monetary (c.c. blood)                         int64
Time (months)                                 int64
whether he/she donated blood in March 2007    int64
dtype: object


#### Outliers:
 - All donors with average donations per month > 0.9

In [37]:
transfusion[transfusion['Frequency (times)']/transfusion['Time (months)'] > 0.9]

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
5,4,4,1000,4,0
47,2,2,500,2,0
48,2,2,500,2,0
49,2,2,500,2,0
510,4,4,1000,4,1


#### Null or Missing Values:

In [38]:
transfusion.isnull().sum()

Recency (months)                              0
Frequency (times)                             0
Monetary (c.c. blood)                         0
Time (months)                                 0
whether he/she donated blood in March 2007    0
dtype: int64

#### Coded Content:

 - 'whether he/she donated blood in March 2007': 1='Yes', 0='No'

# Step 5: Prepare the Data
- Deal with the data as required by the modelling technique
    - **Outliers** (remove or adjust if possible or necessary)
    - **Null** (remove or interpolate if possible or necessary)
    - **Missing Values** (remove or interpolate if possible or necessary)
    - **Coded content** (transform if possible or necessary [str to number or vice-versa])
    - **Normalisation** (if possible or necessary)
    - **Feature Engeneer** (if useful or necessary)

In [ ]:
# Filter/subset/clean the data according to your EDA findings

#### Outliers:

In [47]:
transfusion_culled = transfusion[transfusion['Frequency (times)']/transfusion['Time (months)'] <= 0.9]

#### Normalisation:

In [120]:
from sklearn.preprocessing import normalize

transfusion_normed = transfusion_culled[[var for var in transfusion.columns if var not in ['Monetary (c.c. blood)', 'whether he/she donated blood in March 2007']]]
transfusion_normed = pd.DataFrame(normalize(transfusion_normed))
X = transfusion_normed
y = transfusion_culled['whether he/she donated blood in March 2007']

# Step 6: Modelling
Refer to the Problem and Main Question.
- What are the input variables (features)?
- Is there an output variable (label)?
- If there is an output variable:
    - What is it?
    - What is its type?
- What type of Modelling is it?
    - [ ] Supervised
    - [ ] Unsupervised 
- What type of Modelling is it?
    - [ ] Regression
    - [ ] Classification (binary) 
    - [ ] Classification (multi-class)
    - [ ] Clustering

#### Features:

In [122]:
Features = [var for var in transfusion_culled.columns if var not in ['whether he/she donated blood in March 2007']]
print(Features)

X = transfusion_normed

['Recency (months)', 'Frequency (times)', 'Monetary (c.c. blood)', 'Time (months)']


#### Output:

In [124]:
print('whether he/she donated blood in March 2007')

y = transfusion_culled['whether he/she donated blood in March 2007']

whether he/she donated blood in March 2007


#### Modelling:

 - Supervised binary classification

# Step 7: Split the Data

Need to check for **Supervised** modelling:
- Number of known cases or observations
- Define the split in Training/Test or Training/Validation/Test and their proportions
- Check for unbalanced classes and how to keep or avoid it when spliting

In [128]:
# Split your data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

# Step 8: Define a Model

Define the model and its hyper-parameters.

Consider the parameters and hyper-parameters of each model at each (re)run and after checking the efficiency of a model against the training and test datasets.

In [129]:
# Choose a model or models
### Logistic regression:

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

# Step 9: Fit the Model

In [130]:
# Fit model

fitted_model = model.fit(X_train,y_train)

# Step 10: Verify and Evaluate the Training Model
- Use the **training** data to make predictions
- Check for overfitting
- What metrics are appropriate for the modelling approach used
- For **Supervised** models:
    - Check the **Training Results** with the **Training Predictions** during development
- Analyse, modify the parameters and hyper-parameters and repeat (within reason) until the model does not improve

In [133]:
# Evaluate model against training set
from sklearn.metrics import accuracy_score

accuracy_score(fitted_model.predict(X_train), y_train)

0.765993265993266

# Step 11: Make Predictions and Evaluate the Test Model
**NOTE**: **Do this only after not making any more improvements in the model**.

- Use the **test** data to make predictions
- For **Supervised** models:
    - Check the **Test Results** with the **Test Predictions**

In [134]:
# Evaluate model against test set
accuracy_score(fitted_model.predict(X_test), y_test)

0.8187919463087249

# Step 12: Solve the Problem or Answer the Question
The results of an analysis or modelling can be used:
- As part of a product or process, so the model can make predictions when new input data is available
- As part of a report including text and charts to help understand the problem
- As input for further questions

Yes we can make predictions with over 80% accuracy. it can be used to make blood transfusion more sustainable, by predicting who will need it and how much is needed.

>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



